# OCR

In [1]:
import pytesseract
from pdf2image import convert_from_path

In [2]:
# pdf_path = '../data/patente.pdf'
# pages = convert_from_path(pdf_path)

# # Aplique OCR em cada página
# full_text = ""

# for page_number, page in enumerate(pages):
#     # Converta a página da imagem para texto
#     text = pytesseract.image_to_string(page)
#     full_text += text

# with open('../data/patente.txt', 'w') as file:
#     file.write(full_text)

# Clean Data

In [3]:
import re

def split_document(document):
    # Usar expressão regular para dividir com base em duas ou mais quebras de linha consecutivas (\n\n ou mais)
    return re.split(r'\n\s*\n', document)

# LLM

In [4]:
from langchain_ollama import ChatOllama
from langchain_core.prompts import ChatPromptTemplate

In [5]:
file_path = "../data/patente.txt"

with open(file_path, 'r') as file:
    data = file.read()


In [6]:
prompt = ChatPromptTemplate.from_messages([
  (
    "system",
    # "You are a knowledgeable assistant. Given the following document related to glass, your task is to analyze the document and extract the chemical composition the given glass, detailing the percentage of each element, and listing the key properties associated with the glass. If the document does not contain relevant information regarding the chemical composition or properties, return 'not available'. Here are the contents of the document:\n\n----\n\n{document}."
    "You are a knowledgeable assistant. Given the following document related to glass, your task is to analyze the document and extract the chemical composition the given glass, detailing the percentage of each element, and listing the key properties associated with the glass. If there is no relevant information, do not return a generic aswer, just say 'there is no information'. Here are the contents of the document:\n\n----\n\n{document}."
  ),
  ("human", "{input}")
])

llm = ChatOllama(
  model="llama3.1",
  temperature=0.8,
)

chain = prompt | llm



In [7]:
chunks = split_document(data)

In [18]:
chunk = chunks[17:50]
print(chunk)

['The present invention discloses a glass with high refractive\nindex for fiber optic imaging elements with medium-expan-\nsion and fabrication method therefor, the glass comprising\nthe following components in percentage by weight: SiO, 5-\n9%, AlsO3 0-1%, BsO3 23-28%, CaO 0-3%, BaO 6-12%,\nLa,O3 30-34%, Nb»Os 4-8%, TasOs 0-1%, Y203 0-1%,\nZnO 4-9%, TiO, 4-8%, ZrOz 4-6%, SnO, 0-1%. The present\ninvention further provides a fabrication method for the glass\nwith a high refractive index, comprising: putting raw mate-\nrials quartz sand, aluminum hydroxide, boric acid or boric\nanhydride, calcium carbonate, barium carbonate or barium\nnitrate, lanthanum oxide, niobium oxide, tantalum oxide,\nyttrium oxide, zinc oxide, titanium dioxide, zirconium\noxide and stannic oxide, etc. into a platinum crucible\naccording to the requirement of dosing, melting at a high\ntemperature, cooling and fining, leaking and casting to\nform a glass rod, and then annealing, cooling and chilling\nthe molded gl

In [19]:
# Processar cada parte separadamente e coletar as respostas
# for i, chunk in enumerate(chunks):
ai_msg = chain.invoke({
    "document": chunk,  # Passa o pedaço do documento
    "input": "Respond to me in JSON format. List the chemical compositions of the glass mentioned in the document along with the percentage of each element, and the key properties associated with each glass type. "
})


In [20]:
print(ai_msg.content)

```
{
  "glass_compositions": [
    {
      "name": "High Refractive Index Glass",
      "chemical_composition": {
        "SiO2": "5-9%",
        "Al2O3": "0-1%",
        "B2O3": "23-28%",
        "CaO": "0-3%",
        "BaO": "6-12%",
        "La2O3": "30-34%",
        "Nb2O5": "4.8%",
        "Ta2O5": "0-1%",
        "Y2O3": "0-1%",
        "ZnO": "4.9%",
        "TiO2": "4.8%",
        "ZrO2": "4.6%",
        "SnO2": "0-1%"
      },
      "properties": [
        {
          "property_name": "Refractive Index",
          "property_value": "1.80-1.82"
        },
        {
          "property_name": "Coefficient of Average Linear Thermal Expansion",
          "property_value": "(68 + 5) x 10^-7/°C (30-300°C)"
        },
        {
          "property_name": "Strain Point Temperature",
          "property_value": "> 600°C"
        },
        {
          "property_name": "Devitrification Temperature",
          "property_value": "> 820°C"
        },
        {
          "property_name": "